In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 10
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'':
  os.chdir(r'')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


{"C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap_external.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\codecs.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\aliases.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\__init__.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\utf_8.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\cp1252.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\abc.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\io.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\stat.py

C:\Users\keb7\AppData\Local\Temp\ipykernel_111824\1140549349.py:20: DeprecationWarning:

`set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`



In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import yfinance as yf
import numpy as np

In [3]:
df = pd.read_excel(
    "files/PET_PRI_SPT_S1_D.xls", 
    sheet_name="Data 1",
    skiprows=9,
    header=None,
    parse_dates=[0])
df.columns=['date', 'wti', 'brent']

In [4]:
trace = go.Scatter(
  x=df.date,
  y=df.wti,
  mode="lines",
  hovertemplate="%{x}<br>$%{y:.2f}"
)
fig = go.Figure(trace)

fig.update_yaxes(tickformat=".2f", title = "Spot WTI", tickprefix="$")
fig.update_xaxes(title="Date")
fig.layout.template = "simple_white"
fig.update_layout(margin=dict(l=25, r=25, t=40, b=25))
fig.update_xaxes(title_font_size=18, showgrid=True)
fig.update_yaxes(title_font_size=18, showgrid=True)
fig.update_layout(font_size=18, width=1000)
fig.show()

In [5]:
uso = yf.download("USO", start="1970-01-01", progress=False).reset_index()
uso.Date = pd.to_datetime(uso.Date)

In [6]:
trace = go.Scatter(
  x=uso.Date,
  y=uso.Close,
  hovertemplate="%{x}<br>$%{y:.2f}"
)
fig = go.Figure(trace)

fig.update_yaxes(tickformat=".2f", title="USO", tickprefix="$")
fig.update_xaxes(title="Date")
fig.layout.template = "simple_white"
fig.update_layout(margin=dict(l=25, r=25, t=40, b=25))
fig.update_xaxes(title_font_size=18, showgrid=True)
fig.update_yaxes(title_font_size=18, showgrid=True)
fig.update_layout(font_size=18, width=1000)
fig.show()

In [7]:
uso.Date = [x.strftime("%Y-%m-%d") for x in uso.Date]
df.date = df.date.astype(str)

df = df.merge(uso, left_on="date", right_on="Date")
df["spot"] = df.wti / df.wti.iloc[0]
df["uso"] = df.Close / df.Close.iloc[0]

In [8]:
trace1 = go.Scatter(
  x=df.date,
  y=df.spot,
  name="Spot",
  hovertemplate='$%{y:.2f}<extra></extra>'
)
trace2 = go.Scatter(
  x=df.date,
  y=df.uso,
  name="USO",
  hovertemplate='$%{y:.2f}<extra></extra>'
)
fig = go.Figure()
for trace in [trace1, trace2]:
  fig.add_trace(trace)
fig.update_yaxes(tickformat=".2f", title="Values Starting from $1", tickprefix="$")
fig.update_xaxes(title="Date")
fig.layout.template = "simple_white"
fig.update_layout(margin=dict(l=25, r=25, t=40, b=25))
fig.update_xaxes(title_font_size=18, showgrid=True)
fig.update_yaxes(title_font_size=18, showgrid=True)
fig.update_layout(font_size=18)
fig.update_layout(hovermode="x unified", width=1000)
fig.show()

In [9]:
df = pd.read_csv("files/crude_futures_monthly.csv")
df = df[(df.date>="2006-03-01")&(df.days<=365)]
df = df.sort_values(by=["date", "days"])
import plotly.express as px
fig = px.line(
  df,
  x="days",
  y="settle",
  animation_frame="date",
)
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 200
fig.update_yaxes(range=[0.99*df.settle.min(), 1.01*df.settle.max()])
fig.update_xaxes(range=[0, df.days.max()])
fig.update_yaxes(tickformat=".0f", title="WTI Futures Settle")
fig.update_xaxes(title="Days to Maturity")
fig.layout.template = "simple_white"
fig.update_layout(margin=dict(l=25, r=25, t=40, b=25))
fig.update_xaxes(title_font_size=18, showgrid=True)
fig.update_yaxes(title_font_size=18, showgrid=True)
fig.update_layout(font_size=18, width=1000)

In [10]:
df = yf.download(("SPY", "SPXL", "SPXS"), start="1970-01-01", progress=False)["Adj Close"].dropna()
rets = df.pct_change().reset_index()
df = df.reset_index()

In [11]:
fig = px.scatter(
  rets,
  x="SPY",
  y="SPXL",
  trendline="ols",
  hover_name="Date"
)
fig.update_yaxes(tickformat=".1%", title="SPXL Return")
fig.update_xaxes(tickformat=".1%", title="SPY Return")
fig.layout.template = "simple_white"
fig.update_layout(margin=dict(l=25, r=25, t=40, b=25))
fig.update_xaxes(title_font_size=18, showgrid=True)
fig.update_yaxes(title_font_size=18, showgrid=True)
fig.update_layout(font_size=18, width=1000)
fig.show()

In [12]:
fig = px.scatter(
  rets,
  x="SPY",
  y="SPXS",
  trendline="ols",
  hover_name="Date"
)
fig.update_yaxes(tickformat=".1%", title="SPXS Return")
fig.update_xaxes(tickformat=".1%", title="SPY Return")
fig.layout.template = "simple_white"
fig.update_layout(margin=dict(l=25, r=25, t=40, b=25))
fig.update_xaxes(title_font_size=18, showgrid=True)
fig.update_yaxes(title_font_size=18, showgrid=True)
fig.update_layout(font_size=18, width=1000)
fig.show()

In [13]:
trace = go.Scatter(
  x=df.Date,
  y=df.SPXL,
  name="SPXL",
  hovertemplate="SPXL = $%{y:.2f}<extra></extra>"
)
fig = go.Figure(trace)

fig.update_yaxes(tickformat=".0f", title="SPXL Adj Close", tickprefix="$")
fig.update_xaxes(title="Date")
fig.layout.template = "simple_white"
fig.update_layout(margin=dict(l=25, r=25, t=40, b=25))
fig.update_xaxes(title_font_size=18, showgrid=True)
fig.update_yaxes(title_font_size=18, showgrid=True)
fig.update_layout(font_size=18, width=1000, hovermode="x unified", showlegend=False)
fig.show()

In [14]:
trace = go.Scatter(
  x=df.Date,
  y=df.SPXS,
  name="SPXS",
  hovertemplate="SPXS = $%{y:.2f}<extra></extra>"
)
fig = go.Figure(trace)

fig.update_yaxes(tickformat=".0f", title="SPXS Adj Close", tickprefix="$")
fig.update_xaxes(title="Date")
fig.layout.template = "simple_white"
fig.update_layout(margin=dict(l=25, r=25, t=40, b=25))
fig.update_xaxes(title_font_size=18, showgrid=True)
fig.update_yaxes(title_font_size=18, showgrid=True)
fig.update_layout(font_size=18, width=1000, hovermode="x unified", showlegend=False)
fig.show()

In [15]:
for col in ["SPY", "SPXL", "SPXS"]:
  df[col] = df[col] / df[col].iloc[0]

traces = []
trace = go.Scatter(
  x=df.Date,
  y=df.SPY,
  name="SPY",
  hovertemplate="SPY = $%{y:.2f}<extra></extra>"
)
traces.append(trace)

trace = go.Scatter(
  x=df.Date,
  y=df.SPXL,
  name="SPXL",
  hovertemplate="SPXL = $%{y:.2f}<extra></extra>"
)
traces.append(trace)

trace = go.Scatter(
  x=df.Date,
  y=df.SPXS,
  name="SPXS",
  hovertemplate="SPXS = $%{y:.4f}<extra></extra>"
)
traces.append(trace)

fig = go.Figure()
for trace in traces:
  fig.add_trace(trace)

fig.update_yaxes(tickformat=".0f", title="Accumulation", tickprefix="$")
fig.update_xaxes(title="Date")
fig.layout.template = "simple_white"
fig.update_layout(margin=dict(l=25, r=25, t=40, b=25))
fig.update_xaxes(title_font_size=18, showgrid=True)
fig.update_yaxes(title_font_size=18, showgrid=True)
fig.update_layout(font_size=18, width=1000, hovermode="x unified")
fig.show()

In [16]:
traces = []
trace = go.Scatter(
  x=df.Date,
  y=df.SPY,
  name="SPY",
  hovertemplate="SPY = $%{y:.2f}<extra></extra>"
)
traces.append(trace)

trace = go.Scatter(
  x=df.Date,
  y=df.SPXL,
  name="SPXL",
  hovertemplate="SPXL = $%{y:.2f}<extra></extra>"
)
traces.append(trace)

trace = go.Scatter(
  x=df.Date,
  y=df.SPXS,
  name="SPXS",
  hovertemplate="SPXS = $%{y:.4f}<extra></extra>"
)
traces.append(trace)

fig = go.Figure()
for trace in traces:
  fig.add_trace(trace)

fig.update_yaxes(tickformat=".2f", title="Accumulation", tickprefix="$")
fig.update_xaxes(title="Date")
fig.layout.template = "simple_white"
fig.update_layout(margin=dict(l=25, r=25, t=40, b=25))
fig.update_xaxes(title_font_size=18, showgrid=True)
fig.update_yaxes(title_font_size=18, showgrid=True, type="log")
fig.update_layout(font_size=18, width=1000, hovermode="x unified")
fig.show()

In [17]:
mn = rets.SPY.mean()
sd = rets.SPY.std()
n = 10*252

r = np.random.normal(loc=mn, scale=sd, size=n*1000).reshape(n, 1000)
spy = np.prod(1+r, axis=0)
spxl = np.prod(1+3*r, axis=0)
trace = go.Scatter(
    x=spy,
    y=spxl,
    mode="markers",
    hovertemplate="SPY = $%{x:.2f}<br>SPXL = $%{y:.2f}<extra></extra>"
)
fig = go.Figure(trace)
fig.update_yaxes(tickformat=",.0f", title="Simulated SPXL Accumulation", tickprefix="$")
fig.update_xaxes(tickformat=".0f", title="Simulated SPY Accumulation", tickprefix="$")
fig.layout.template = "simple_white"
fig.update_layout(margin=dict(l=25, r=25, t=40, b=25))
fig.update_xaxes(title_font_size=18, showgrid=True)
fig.update_yaxes(title_font_size=18, showgrid=True)
fig.update_layout(font_size=18, width=1000)
fig.show()